In [ ]:
import json
import os
import pandas as pd
from main import show_arch_comp_results, get_results_from_folder
import matplotlib.pyplot as plt
from utils.customtypes import Circuit

# plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams.update({'font.size': 14})


palette = {
    "Vibrant Blue": "#3485fd",   # Primary brand color, deep and saturated
    "Sun Yellow": "#ffcb2f",     # Brightest, highest contrast
    "Royal Purple": "#8660cb",   # Deep, rich secondary color
    "Lush Green": "#4ab563",     # Success/Nature
    "Coral Red": "#e25563",      # Error/Warning, slightly softened
    "Ocean Turquoise": "#44d2a8",  # Clear, bright accent
    "Sunset Orange": "#fd933a",   # Warm and vibrant
    "Sky Light Blue": "#3cb1c3"   # Tertiary cool color
}

# Embedding Size Comparison

In [ ]:
data = show_arch_comp_results('../trained/architecture')
data = sorted([(int(k.split('_')[1]),v) for k,v in data.items()])
data = data[1:] # Remove first value as it makes it difficult to see the data
x = [d[0] for d in data]
y = [d[1] for d in data]
print(data)
plt.scatter(x, y, marker="x")
plt.plot(x, y)

plt.xscale('log', base=2)
plt.xticks(x, [str(t) for t in x])

# plt.grid()
plt.xlabel("Embedding size (H), log scale")
plt.ylabel("Normalized cost")


In [ ]:
results = get_results_from_folder('../trained/architecture')
results = sorted([(int(k.split('_')[1]),v) for k,v in results.items()])[1:]

# plt.grid()

for (k,v) in results:
  cost = v['validation_cost']
  plt.plot(list(range(len(cost))), cost, label=k)

plt.legend(loc="best")
plt.xlabel("Train iteration")
plt.ylabel("Normalized cost")

# Depth comparison

In [ ]:
data = show_arch_comp_results('../trained/shape')
data = {k.split('_')[1]:v for k,v in data.items()}

# Extract keys and values
labels = ['direct', 'double', 'triple', 'end']
values = list(data[l] for l in labels)

# Create bar plot
bars = plt.bar(labels, values)
plt.ylim((1.4,1.6))
plt.ylabel("Normalized cost")

# plt.xticks(x, [str(t) for t in x])

In [ ]:
results = get_results_from_folder('../trained/shape')
results = {k.split('_')[1]:v for k,v in results.items()}

# plt.grid()

for l in labels:
  cost = results[l]['validation_cost']
  plt.plot(list(range(len(cost))), cost, label=l)

plt.legend(loc="best")
plt.xlabel("Train iteration")
plt.ylabel("Normalized cost")

# Training DA

In [ ]:
with open('../trained/da_v3/train_data.json', 'r') as f:
    results = json.load(f)

cost = results['validation_cost']
plt.plot(list(range(len(cost))), cost)
plt.grid()
plt.ylim((1.2,2))
plt.xlabel("Train iteration")
plt.ylabel("Normalized cost")

# SOTA Comparison

In [ ]:
def format_name(name):
    if len(name) < 4:
        return name.upper()
    return name.replace('_', ' ').title()

def show_res(what:str, n_qubits: int, get_mine: bool):
    if get_mine:
        results = pd.read_csv(f'../data/my_{what}_{n_qubits}.csv', index_col="circuit")
    else:
        results = pd.read_csv(f'../data/sota_{what}_{n_qubits}.csv', index_col="circuit")

    # Normalize results
    with open(f'../data/all_{n_qubits}.json', 'r') as f:
        data = json.load(f)
    n_qubits = data['n_qubits']

    if what == 'cost':
        for (name, slices) in data['circuits'].items():
            n_gates = Circuit(slice_gates=slices, n_qubits=n_qubits).n_gates_norm
            results.loc[name] = results.loc[name] / (n_gates + 1)
    else:
        results /= 60


    # Avg random circuit rows
    mask = results.index.str.startswith("random")
    random_mean = results[mask].mean().to_frame().T
    random_mean.index = ["random_avg"]
    results = pd.concat([results[~mask], random_mean])


    algorithms = results.columns
    experiments = [format_name(exp) for exp in results.index]

    x = list(range(len(experiments)))
    width = 0.8 / len(algorithms)

    plt.figure(figsize=(10, 5))

    alg_colors = {
        'russo': 'Sunset Orange',
        'hqa': 'Sky Light Blue',
        'da_sequential': 'Ocean Turquoise',
        'da_parallel': 'Lush Green',
    }

    for i, alg in enumerate(algorithms):
        x_ = [xi + i * width for xi in x]
        plt.bar(x_, results[alg], width, label=format_name(alg), color=palette[alg_colors[alg]])

    x_ = [xi + width * (len(algorithms) - 1) / 2 for xi in x]
    plt.xticks(
        x_,
        experiments,
        rotation=45
    )

    plt.ylabel("Normalized Cost" if what == 'cost' else 'Execution Time [mins]')
    plt.legend()
    plt.tight_layout()
    plt.title(f'{n_qubits} Qubits')
    plt.show()

    print(results)


get_mine = True
for what in ['cost', 'time']:
    for n_qubits in [50, 100]:
    # for n_qubits in [50]:
        show_res(what, n_qubits, get_mine)

In [13]:
import csv

fname = '/mnt/kaliberai/victorc/cole/trained/da_v2/memory_log.txt'

total_mem = {}

with open(fname, 'r') as f:
    reader = csv.reader(f)
    rows = list(reader)

    for (row_i, row_ipp) in zip(rows[:-1],rows[1:]):
        dmem = float(row_ipp[0]) - float(row_i[0])
        key = f'line {row_i[2]} in func {row_i[3]} - line {row_ipp[2]} in func {row_ipp[3]}'
        if key in total_mem.keys():
            total_mem[key] += dmem
        else:
            total_mem[key] = dmem

total_mem = sorted(total_mem.items(), key=lambda x: x[1], reverse=True)

for k,v in total_mem:
    print(f"{k}: {v}")

line 239 in func _allocate_sequential - line 249 in func _allocate_sequential: 9.575371000000223
line 559 in func train - line 509 in func train: 1.8634229999999974
line 249 in func _allocate_sequential - line 559 in func train: 0.6719850000000012
line 249 in func _allocate_sequential - line 239 in func _allocate_sequential: 0.4802489999997761
line 217 in func _allocate_sequential - line 239 in func _allocate_sequential: 0.019879000000001423
line 249 in func _allocate_sequential - line 217 in func _allocate_sequential: 0.00266099999999847
line 509 in func train - line 217 in func _allocate_sequential: -0.08454499999999698
